In [1]:
import polars as pl

In [2]:
df = pl.DataFrame({'keys': list('aab'), 'vals': [10, 7, 1]})
df

keys,vals
str,i64
"""a""",10
"""a""",7
"""b""",1


In [3]:
df.groupby('keys', maintain_order=True).agg([
    pl.col('vals').map(lambda s: s.shift()).alias('shiftmap'),
    pl.col('vals').shift().alias('shiftexpr')])

keys,shiftmap,shiftexpr
str,list[i64],list[i64]
"""a""","[null, 10]","[null, 10]"
"""b""",[7],[null]


In [4]:
(df
 .groupby('keys', maintain_order=True)
 .agg([
     pl.col('vals').apply(lambda s: s.shift()).alias('shift_map'),
     pl.col('vals').shift().alias('shift_expr')]))

keys,shift_map,shift_expr
str,list[i64],list[i64]
"""a""","[null, 10]","[null, 10]"
"""b""",[null],[null]


In [5]:
def add_counter(val: int) -> int:
    global counter
    counter += 1
    return counter + val

In [6]:
counter = 0
df.select([
    pl.col('vals').apply(add_counter).alias('ctr_apply'),
    (pl.col('vals') + pl.arange(1, pl.count() + 1)).alias('ctr_expr')])

ctr_apply,ctr_expr
i64,i64
11,11
9,9
4,4


In [8]:
df.select([
    pl.struct(['keys', 'vals']).apply(lambda x: len(x['keys']) + x['vals']).alias('app'),
    (pl.col('keys').str.lengths() + pl.col('vals')).alias('expr')])

app,expr
i64,i64
11,11
8,8
2,2
